# EleMLDS - Tutorial Exercise 4: Linear Discriminants
In this exercise, you will implement linear discriminant models to classify some toy data.

Make sure to replace all parts that say
```python
# YOUR CODE HERE
raise NotImplementedError()
```

Happy coding!

# Q1: Least Squares Linear Classifier
We start off with a simple least-squares classifier.
This model learns the weights and bias of a linear discriminant function $y(\mathbf{x})$:

$$y(\mathbf{x}) = \mathbf{w}^T\mathbf{x} + w_0$$

Train a least-squares linear classifier on synthetic 2D training data and evaluate it on the training and test set.
You should have extracted the files containing the training and test data together with this notebook.

We already implemented the dataloading, plotting, and evaluation code. Feel free to take a look at them before continuing with the exercise further below.
You will need to write two functions:
- `leastSquares` trains a linear discriminant using the least-squares objective.
- `linclass` applies a linear classifier on some data and returns the predictions.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def load_data(name):
    data = np.load(f"{name}.npz")
    return {s: data[s] for s in ("data", "labels")}


def plot_contour(fun):
    # make a regular grid over the whole plot
    x, y = np.linspace(*plt.xlim(), 200), np.linspace(*plt.ylim(), 200)
    xx, yy = np.meshgrid(x, y)
    grid = np.c_[xx.ravel(), yy.ravel()]
    # evaluate function on grid
    res = fun(grid).reshape(*xx.shape)

    # plot contour lines (=decision boundary) and filled contours over whole plot
    plt.contourf(xx, yy, res, colors=["yellow", "blue"], alpha=0.2)
    plt.contour(xx, yy, res, levels=1, colors="k")


def plot_(data, labels, params=None, title=None, basis_fun=None):
    # Plot the data points and the decision line
    plt.subplot()
    if title:
        plt.title(title)

    class1, class2 = data[labels > 0], data[labels < 0]
    plt.scatter(*class1.T, c="blue", marker="x")
    plt.scatter(*class2.T, c="orange", marker="o")

    if params:
        w, b = params
        xmax = data[:, 0].max(0)
        xmin = data[:, 0].min(0)
        # Quick and hacky way to fix the y-axis limits
        plt.ylim(plt.ylim())

        if basis_fun:
            # evaluate decision function on grid and plot contours
            plot_contour(lambda grid: linclass(w, b, basis_fun(grid)))
        else:
            # just plot a line
            y = lambda x: -(w[0] * x + b) / w[1]
            plt.plot([xmin, xmax], [y(xmin), y(xmax)], c="k")

    plt.show()

## Q1.1
Implement the `leastSquares` function.
It should train a least-squares classifier based on the data matrix $\mathbf{X}$ and its class label vector $\mathbf{t}$.
As output, it produces the linear classifier weight vector $\mathbf{w}$ and bias $b$.

In [ ]:
def leastSquares(data, label):
    # Minimize the sum-of-squares error
    #
    # INPUT:
    # data        : Training inputs  (num_samples x dim)
    # label       : Training targets- 1D array of length {num_samples}
    #
    # OUTPUT:
    # weights     : weights- 1D array of length {dim}
    # bias        : bias term (scalar)

    

    # YOUR CODE HERE
    raise NotImplementedError()

    return weight, bias

## Q1.2
Implement the function `linclass` that classifies a data matrix $\mathbf{X}$ based on a trained linear classifier given by $\mathbf{w}$ and $b$.
Remember that we expect classification results to be either 1 or -1.

In [ ]:
def linclass(weight, bias, data):
    # Apply a linear classifier
    #
    # INPUT:
    # weight      : weights                1D array of length {dim}
    # bias        : bias term              (scalar)
    # data        : Input to be classified (num_samples x dim)
    #
    # OUTPUT:
    # class_pred       : Predicted class (+-1) values- 1D array of length {num_samples}

    # YOUR CODE HERE
    raise NotImplementedError()

    return class_pred

## Q1.3
Run the cells below to train and evaluate a linear classifier on the provided data.
Analyze the classification plots for both the datasets. Are the sets optimally classified?

In [ ]:
train = load_data("lc_train")
test = load_data("lc_test")


def evaluate(data, label, params, title, basis_fun=None):
    pred = linclass(weight, bias, basis_fun(data) if basis_fun is not None else data)
    acc = (pred == label).mean()
    print(f"Accuracy on {title}: {acc:.5f}")
    plot_(data, label, params, title, basis_fun)

In [ ]:
weight, bias = leastSquares(train["data"], train["labels"])

# Evaluate on the train set
evaluate(train["data"], train["labels"], (weight, bias), "Train Set")

# Evaluate on the test set
evaluate(test["data"], test["labels"], (weight, bias), "Test Set")

## Q1.4
Now we add some outliers to the data.
Run the cells below to train and evaluate the classifier on the augmented data.

Again, analyze the classification plots for both datasets. What do you notice?

In [ ]:
outlier_train = {
    "data": np.append(train["data"], [[1.5, -0.4], [1.45, -0.35]], axis=0),
    "labels": np.append(train["labels"], [[-1], [-1]]),
}

weight, bias = leastSquares(outlier_train["data"], outlier_train["labels"])

# Evaluate on the train set
evaluate(
    outlier_train["data"],
    outlier_train["labels"],
    (weight, bias),
    "Train Set with Outliers",
)

# Evaluate on the test set
evaluate(test["data"], test["labels"], (weight, bias), "Test Set")

# Q2: Basis Functions
Now we will implement polynomial basis functions and use them to classifiy non-linearly separable data.
Our basis functions will have the following form:

To start, implement $\phi_2(\mathbf{x})$, that is, a second degree polynomial basis function, for two-dimensional data:

$$
\phi_2(\mathbf{x}) = (1, x_1, x_2, x_1^2, x_1 x_2, x_2^2)^T
$$

In [ ]:
def poly_two(x):
    # YOUR CODE HERE
    raise NotImplementedError()


w = np.array([0.25, -0.5, -0.5, 0, 0, 1])
plot_contour(lambda x: linclass(w, 0, poly_two(x)))
plt.show()

Now train a linear discriminant using this basis function.

In [ ]:
train_poly = poly_two(outlier_train["data"])
test_poly = poly_two(test["data"])

weight, bias = leastSquares(train_poly, outlier_train["labels"])

# Evaluate on the train set
evaluate(
    outlier_train["data"],
    outlier_train["labels"],
    (weight, bias),
    "Train Set",
    poly_two,
)

# Evaluate on the test set
evaluate(test["data"], test["labels"], (weight, bias), "Test Set", poly_two)